In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split


In [ ]:
"""import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import scipy.stats as stats
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import skew, kurtosis, probplot
from statsmodels.stats.stattools import jarque_bera
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.stats import shapiro
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from lightgbm import LGBMRegressor
import lightgbm as lgb
from lightgbm import early_stopping
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error, r2_score, make_scorer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
import plotly.graph_objects as go
import joblib"""

In [ ]:
#import warnings
#warnings.simplefilter(action='ignore', category=Warning)

In [1]:
!pip install mlflow
!pip install --upgrade jinja2
!pip install --upgrade Flask
!pip install setuptools

  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
   ---------------------------------------- 0.0/28.2 MB ? eta -:--:--
   ----- ---------------------------------- 3.7/28.2 MB 18.2 MB/s eta 0:00:02
   ---------- ----------------------------- 7.1/28.2 MB 16.8 MB/s eta 0:00:02
   ----------------- ---------------------- 12.1/28.2 MB 19.4 MB/s eta 0:00:01
   ------------------------- -------------- 17.8/28.2 MB 21.2 MB/s eta 0:00:01
   --------------------------------- ------ 23.9/28.2 MB 22.5 MB/s eta 0:00:01
   ---------------------------------------  28.0/28.2 MB 23.4 MB/s eta 0:00:01
   ---------------------------------------- 28.2/28.2 MB 22.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
   ---------------------------------------- 6.1/6.1 MB 28.8 MB/s eta 0:00:00
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
   ---------------------------------------- 0.0/681.0 kB ? eta -:--:--
   --------------------------

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openai 1.68.0 requires numpy>=2.0.2, but you have numpy 1.26.4 which is incompatible.
streamlit 1.34.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.4 which is incompatible.
streamlit 1.34.0 requires tenacity<9,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 3.1.4
    Uninstalling Jinja2-3.1.4:
      Successfully uninstalled Jinja2-3.1.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.34.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.4 which is incompatible.
streamlit 1.34.0 requires tenacity<9,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.0.3
    Uninstalling Werkzeug-3.0.3:
      Successfully uninstalled Werkzeug-3.0.3
  Attempting uninstall: blinker
    Found existing installation: blinker 1.6.2
    Uninstalling blinker-1.6.2:
      Successfully uninstalled blinker-1.6.2
  Attempting uninstall: Flask
    Found existing installation: Flask 3.0.3
    Uninstalling Flask-3.0.3:
      Successfully uninstalled Flask-3.0.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.34.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.4 which is incompatible.
streamlit 1.34.0 requires tenacity<9,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Plan d’Action
Démarrer le serveur MLflow (tu l’as déjà fait)
Créer l’expérience MLflow adaptée à ton projet
Enregistrer les modèles et métriques XGBoost et LGBM dans MLflow
Tester que tout fonctionne

### Démarrer MLflow

In [2]:

# starts an MLflow server locally.
!mlflow server --host 127.0.0.1 --port 8080

^C


### Configurer MLflow pour notre projet

In [3]:
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

# Connexion au serveur MLflow
mlflow.set_tracking_uri("http://127.0.0.1:8080")

# Créer une expérience MLflow spécifique à ton projet
experiment_name = "Forecasting_Energie"
mlflow.set_experiment(experiment_name)

client = MlflowClient()
experiment = client.get_experiment_by_name(experiment_name)

if experiment is None:
    experiment_id = client.create_experiment(experiment_name)
else:
    experiment_id = experiment.experiment_id

print(f"Expérience MLflow créée: {experiment_name} (ID: {experiment_id})")


2025/03/21 17:28:00 INFO mlflow.tracking.fluent: Experiment with name 'Forecasting_Energie' does not exist. Creating a new experiment.


Expérience MLflow créée: Forecasting_Energie (ID: 373507975995929459)


In [21]:
df = pd.read_csv("C:/Users/PC/Projet_MLOPS/data/df_final_ML2.csv")


### Conversions 

In [22]:
#Convertir la colonne Date en datetime
df["Date"] = pd.to_datetime(df["Date"])

#Convertir la colonne Code INSEE région en object
df['Code INSEE région'] = df['Code INSEE région'].astype('object')

#Convertir la colonne Loi energie climat 2019 en integer

df['Loi energie climat 2019'] = df['Loi energie climat 2019'].astype('int')

### Feature engineering 

En plus des variables temporelles ajoutés dans la partie data management (jour, mois, année, sinus, cosinus, etc), nous allons également procéder à la construction de lags

In [23]:
# Ajout des lags pour la consommation d'énergie jusqu'à 90 jours
for lag in [1, 7, 30, 60, 90, 365]:  # Lags de 1, 7, 30, 60 et 90 jours
    df[f'lag_{lag}_Consommation'] = df['Consommation (MWh)'].shift(lag)

# Ajout des lags pour les variables climatiques jusqu'à 90 jours
for lag in [1, 7, 30, 60, 90]:
    df[f'lag_{lag}_TMin'] = df['TMin (°C)'].shift(lag)
    df[f'lag_{lag}_TMax'] = df['TMax (°C)'].shift(lag)
    df[f'lag_{lag}_TMoy'] = df['TMoy (°C)'].shift(lag)
    df[f'lag_{lag}_Vitesse_vent'] = df['Vitesse du vent à 100m (m/s)'].shift(lag)
    df[f'lag_{lag}_Rayonnement'] = df['Rayonnement solaire global (W/m2)'].shift(lag)

# Supprimer les valeurs manquantes créées par les lags
df = df.dropna()

In [24]:
df = df.sort_values(['Code INSEE région', 'Date'])

# Définir les fenêtres de rolling
windows = [7, 30, 90]  # 7 jours, 30 jours, 90 jours

for window in windows:
    df[f'Consommation_rolling_mean_{window}'] = df.groupby('Code INSEE région')['Consommation (MWh)'].transform(lambda x: x.rolling(window=window, min_periods=1).mean())

#Gestion des NA : 

# Remplir les NaN avec la méthode de forward fill **par région**
df['Consommation_rolling_mean_7'] = df.groupby('Code INSEE région')['Consommation_rolling_mean_7'].fillna(method='ffill')
df['Consommation_rolling_mean_30'] = df.groupby('Code INSEE région')['Consommation_rolling_mean_30'].fillna(method='ffill')
df['Consommation_rolling_mean_90'] = df.groupby('Code INSEE région')['Consommation_rolling_mean_90'].fillna(method='ffill')


C:\Users\PC\AppData\Local\Temp\ipykernel_15764\13998302.py:12: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  df['Consommation_rolling_mean_7'] = df.groupby('Code INSEE région')['Consommation_rolling_mean_7'].fillna(method='ffill')
C:\Users\PC\AppData\Local\Temp\ipykernel_15764\13998302.py:12: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Consommation_rolling_mean_7'] = df.groupby('Code INSEE région')['Consommation_rolling_mean_7'].fillna(method='ffill')
C:\Users\PC\AppData\Local\Temp\ipykernel_15764\13998302.py:13: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single val

### Gestion des outliers

In [25]:
# Calculer Q1, Q3 et l'IQR pour la target
Q1 = df['Consommation (MWh)'].quantile(0.25)
Q3 = df['Consommation (MWh)'].quantile(0.75)
IQR = Q3 - Q1

# Identifier les outliers dans la consommation
outliers = (df['Consommation (MWh)'] < (Q1 - 1.5 * IQR)) | (df['Consommation (MWh)'] > (Q3 + 1.5 * IQR))

# Supprimer les lignes avec outliers dans la target
df_cleaned = df[~outliers]

### Train / test split et transformations

In [26]:
df_transformed = df_cleaned.copy() #copier le data set pour ne pas écraser les modifications

In [27]:
# Séparer la target (y) des features (X)
df_transformed = df_transformed.sort_values(by=['Date', 'Code INSEE région']).\
  reset_index(drop=True)

X = df_transformed[['Région','Thermique (MWh)', 
       'Hydraulique (MWh)', 'Bioénergies (MWh)', 
       'TMoy (°C)', 'Semaine_cos','Semaine_sin', 'lag_7_Consommation', 'lag_30_Consommation', 'lag_365_Consommation', 
                    'lag_7_TMoy', 'Consommation_rolling_mean_7']]

y = df_transformed['Consommation (MWh)']

# Split train/test (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)#shuffle = False pour conserver l'ordre chronologique

In [28]:
from sklearn.preprocessing import OneHotEncoder
#Encodage de la variable Région
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit uniquement sur X_train et transformer X_train
X_train_encoded = encoder.fit_transform(X_train[['Région']])
X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out(['Région']), index=X_train.index)

# Transformer X_test (sans refit)
X_test_encoded = encoder.transform(X_test[['Région']])
X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out(['Région']), index=X_test.index)

# Concaténer les colonnes encodées avec les autres features
X_train = pd.concat([X_train.drop(columns=['Région']), X_train_encoded_df], axis=1)
X_test = pd.concat([X_test.drop(columns=['Région']), X_test_encoded_df], axis=1)

In [29]:
# Concaténer X_train et y_train pour faciliter l'analyse
df_train = X_train[['Région_Auvergne-Rhône-Alpes',
       'Région_Bourgogne-Franche-Comté', 'Région_Bretagne',
       'Région_Centre-Val de Loire', 'Région_Grand Est',
       'Région_Hauts-de-France', 'Région_Normandie',
       'Région_Nouvelle-Aquitaine', 'Région_Occitanie',
       'Région_Pays de la Loire', "Région_Provence-Alpes-Côte d'Azur",
       'Région_Île-de-France']].copy()
df_train['Consommation (MWh)'] = y_train  

# Calculer la corrélation de chaque feature avec la target
correlations = df_train.corr()['Consommation (MWh)'].drop('Consommation (MWh)')  

# Trier par ordre décroissant d'importance absolue
correlations = correlations.abs().sort_values(ascending=False)

# Afficher les variables les plus corrélées
print(correlations)


Région_Auvergne-Rhône-Alpes          0.452152
Région_Île-de-France                 0.438790
Région_Centre-Val de Loire           0.353461
Région_Bourgogne-Franche-Comté       0.305078
Région_Bretagne                      0.284825
Région_Hauts-de-France               0.215121
Région_Pays de la Loire              0.198573
Région_Normandie                     0.188714
Région_Grand Est                     0.132741
Région_Nouvelle-Aquitaine            0.098058
Région_Provence-Alpes-Côte d'Azur    0.045452
Région_Occitanie                     0.004412
Name: Consommation (MWh), dtype: float64


In [30]:
#Garder uniquement les régions avec une corrélation supérieur à 0.30
X_train = X_train.drop(columns=['Région_Bretagne',
        'Région_Grand Est','Région_Hauts-de-France', 'Région_Normandie',
       'Région_Nouvelle-Aquitaine', 'Région_Occitanie',
       'Région_Pays de la Loire', "Région_Provence-Alpes-Côte d'Azur"
       ])

X_test = X_test.drop(columns=['Région_Bretagne',
        'Région_Grand Est','Région_Hauts-de-France', 'Région_Normandie',
       'Région_Nouvelle-Aquitaine', 'Région_Occitanie',
       'Région_Pays de la Loire', "Région_Provence-Alpes-Côte d'Azur"
       ])

In [31]:
from sklearn.preprocessing import StandardScaler
# Colonnes numériques à standardiser
normalize_cols = ['Thermique (MWh)', 
       'Hydraulique (MWh)', 'Bioénergies (MWh)', 
       'TMoy (°C)',  'lag_7_Consommation', 'lag_30_Consommation', 'lag_365_Consommation', 
                    'lag_7_TMoy', 'Consommation_rolling_mean_7']

# Initialisation des scalers
scaler_X = StandardScaler()

# Standardiser X_train 
X_train[normalize_cols] = scaler_X.fit_transform(X_train[normalize_cols])

# Appliquer la transformation sur  X_test (sans refit)
X_test[normalize_cols] = scaler_X.transform(X_test[normalize_cols])


In [32]:
X_train.to_csv("X_train_vf2.csv", index=False)

### LightGBM

In [36]:
import mlflow
import mlflow.sklearn
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis, jarque_bera, shapiro
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Définition du modèle LightGBM
model_lgb = lgb.LGBMRegressor(random_state=42)

# Hyperparamètres à tester avec GridSearchCV
param_grid_lgb = {
    'n_estimators': [200, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'num_leaves': [31, 40, 50],
    'colsample_bytree': [0.7, 0.8, 1.0],
    'subsample': [0.8, 1.0]
}

# GridSearchCV pour la recherche des meilleurs hyperparamètres
grid_search_lgb = GridSearchCV(estimator=model_lgb, param_grid=param_grid_lgb, 
                                cv=3, scoring='neg_mean_absolute_error', 
                                n_jobs=-1, verbose=1)


# Initialiser MLflow
mlflow.set_experiment("LightGBM_Experiment")

with mlflow.start_run():
    
    # Entraîner le modèle avec GridSearch
    grid_search_lgb.fit(X_train, y_train)
    
    # Afficher les meilleurs paramètres trouvés
    print(f"Meilleurs paramètres : {grid_search_lgb.best_params_}")
    
    # Log des meilleurs paramètres
    mlflow.log_params(grid_search_lgb.best_params_)

    # Récupérer le meilleur modèle et hyperparamètres
    best_model_lgb = grid_search_lgb.best_estimator_
    best_params = grid_search_lgb.best_params_

    # Log des hyperparamètres trouvés par GridSearch
    mlflow.log_params(best_params)

    # Validation croisée avec TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=3)
    mae_scores, mse_scores, mape_scores, r2_scores = [], [], [], []

    for train_index, test_index in tscv.split(X_train):
        X_train_iter, X_test_iter = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_iter, y_test_iter = y_train.iloc[train_index], y_train.iloc[test_index]

        # Entraîner le meilleur modèle
        best_model_lgb.fit(X_train_iter, y_train_iter)

        # Prédictions
        y_pred = best_model_lgb.predict(X_test_iter)

        # Calcul des métriques
        mae = mean_absolute_error(y_test_iter, y_pred)
        mse = mean_squared_error(y_test_iter, y_pred)
        mape = np.mean(np.abs((y_test_iter - y_pred) / y_test_iter)) * 100
        r2 = r2_score(y_test_iter, y_pred)

        mae_scores.append(mae)
        mse_scores.append(mse)
        mape_scores.append(mape)
        r2_scores.append(r2)

    # Moyennes des métriques
    avg_mae, avg_mse, avg_mape, avg_r2 = np.mean(mae_scores), np.mean(mse_scores), np.mean(mape_scores), np.mean(r2_scores)

    # Log des métriques dans MLflow
    mlflow.log_metric("MAE_CV", avg_mae)
    mlflow.log_metric("MSE_CV", avg_mse)
    mlflow.log_metric("MAPE_CV", avg_mape)
    mlflow.log_metric("R2_CV", avg_r2)

    # Sauvegarde du meilleur modèle
    joblib.dump(best_model_lgb, 'best_model_lgbm.pkl')
    mlflow.sklearn.log_model(best_model_lgb, "model")

    # Prédictions sur l'ensemble train et test
    y_pred_train = best_model_lgb.predict(X_train)
    y_pred_test = best_model_lgb.predict(X_test)

    # Calcul des métriques finales
    mae_train = mean_absolute_error(y_train, y_pred_train)
    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)
    mape_train = np.mean(np.abs((y_train - y_pred_train) / y_train)) * 100

    mae_test = mean_absolute_error(y_test, y_pred_test)
    mse_test = mean_squared_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)
    mape_test = np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100

    # Log des métriques finales
    mlflow.log_metric("MAE_Train", mae_train)
    mlflow.log_metric("MSE_Train", mse_train)
    mlflow.log_metric("MAPE_Train", mape_train)
    mlflow.log_metric("R2_Train", r2_train)

    mlflow.log_metric("MAE_Test", mae_test)
    mlflow.log_metric("MSE_Test", mse_test)
    mlflow.log_metric("MAPE_Test", mape_test)
    mlflow.log_metric("R2_Test", r2_test)

    print(f"MAE Train: {mae_train}, MAE Test: {mae_test}")
    print(f"MSE Train: {mse_train}, MSE Test: {mse_test}")
    print(f"R² Train: {r2_train}, R² Test: {r2_test}")
    print(f"MAPE Train: {mape_train}%, MAPE Test: {mape_test}%")

    # Fin de l'expérience MLflow
    mlflow.end_run()


Fitting 3 folds for each of 324 candidates, totalling 972 fits
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003921 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2386
[LightGBM] [Info] Number of data points in the train set: 26121, number of used features: 15
[LightGBM] [Info] Start training from score 104592.119693
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

2025/03/21 18:19:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


MAE Train: 2822.8491841148943, MAE Test: 4880.967417750925
MSE Train: 15030236.284654276, MSE Test: 46515353.64367926
R² Train: 0.9928975646251125, R² Test: 0.9752328235232935
MAPE Train: 3.101979232314049%, MAPE Test: 6.076982827754325%
🏃 View run polite-fowl-912 at: http://127.0.0.1:8080/#/experiments/333425846085939848/runs/59a2a15d555141f683a4c48910bda7aa
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/333425846085939848
